In [33]:
import pandas as pd
import psycopg2
import pandas.io.sql as psql
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Point
from statsmodels.distributions.empirical_distribution import ECDF
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.snowball import SnowballStemmer
from langdetect import detect
import re
import emoji
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import plot_confusion_matrix
%matplotlib inline

ImportError: No module named 'lightgbm'

In [22]:
names = [
    "user",
    "latitude",
    "longitude",
    "tweets"
]

users = pd.read_csv("../location/datasets/na/user_info.dev", sep='	', header=None, names=names)

In [23]:
users.head()

,user,latitude,longitude,tweets
0,_AgusO,25.756750,-80.373872,RT @JayTuLive: I love a girl who knows exactly...
1,nativadomar,38.945035,-77.442809,"Getting beyonce hair today! Uh uh oh, uh uh uh..."
2,socks88,43.693552,-79.390902,I'm at Goodlife Fitness (250 Davisville Avenue...
3,Glamazon_Blanco,30.346326,-81.552688,@tiicklemeejayy ;) xoxo ||| @kgurlballin @Chef...
4,Chamerlik,42.064860,-87.938200,Kurt Vonnegut – Writing 101 | Luke James | Blo...


In [24]:
def split_tweets(row):
    return list(map(lambda tweet: tweet.strip(), row["tweets"].split('|||')))

def lowercase(row, column):
    return row[column].lower()

In [25]:
users['tweets'] = users.apply(split_tweets, axis=1)
users["user"] = users.apply(lowercase, axis=1, column="user")

In [26]:
users.head()

,user,latitude,longitude,tweets
0,_aguso,25.756750,-80.373872,[RT @JayTuLive: I love a girl who knows exactl...
1,nativadomar,38.945035,-77.442809,"[Getting beyonce hair today! Uh uh oh, uh uh u..."
2,socks88,43.693552,-79.390902,[I'm at Goodlife Fitness (250 Davisville Avenu...
3,glamazon_blanco,30.346326,-81.552688,"[@tiicklemeejayy ;) xoxo, @kgurlballin @ChefKa..."
4,chamerlik,42.064860,-87.938200,[Kurt Vonnegut – Writing 101 | Luke James | Bl...


In [34]:
users.shape

(9998, 4)

In [43]:
def users_mentions(row):
    for tweet in row["tweets"]:
        return re.findall(r'@\w+', tweet)

def users_hashtags(row):
    for tweet in row["tweets"]:
        return re.findall(r'#\w+', tweet)    

In [44]:
users["mentions"] = users.apply(users_mentions, axis=1)
users["hashtags"] = users.apply(users_hashtags, axis=1)

In [45]:
users

,user,latitude,longitude,tweets,mentions,hashtags
0,_aguso,25.756750,-80.373872,[RT @JayTuLive: I love a girl who knows exactl...,[@JayTuLive],[]
1,nativadomar,38.945035,-77.442809,"[Getting beyonce hair today! Uh uh oh, uh uh u...",[],[]
2,socks88,43.693552,-79.390902,[I'm at Goodlife Fitness (250 Davisville Avenu...,[],[]
3,glamazon_blanco,30.346326,-81.552688,"[@tiicklemeejayy ;) xoxo, @kgurlballin @ChefKa...",[@tiicklemeejayy],[]
4,chamerlik,42.064860,-87.938200,[Kurt Vonnegut – Writing 101 | Luke James | Bl...,[],[]
5,riflekind,36.436475,-77.097888,[im fairly certain that they're not speaking e...,[],[]
6,atlreokingpin,33.956076,-83.990010,"[AGAIN! (@ Kani House) http://t.co/XeS2jdKO, I...",[],[]
7,absvargas,43.841100,-79.558050,"[November &lt;3, 'RT @DarielMusic: People gonn...",[],[]
8,slacksabnk,39.727560,-104.920780,"[The after party is even more fun., @KylieAlex...",[],[]
9,hileykarvey,36.067077,-115.279328,[Kitchen is packing.... Thanks to @JillClark3 ...,"[@JillClark3, @brittjaarsma]",[]
